# Run the Amigo pipeline

In [1]:
import jax
from amigo.pipelines import process_calslope

jax.config.update("jax_enable_x64", True)

path = "../../data"
programs = ["GO1843"]

for program in programs:
    process_calslope(f"{path}/uncal/{program}/", f"{path}/calslope/{program}/")

Cleaning existing directory
Running calslope processing...


  0%|          | 0/6 [00:00<?, ?it/s]

Done

